# 4.시뮬레이션

구간 정해서 시작 전까지 초기train -> 일주일치 시뮬 -> 구간업뎃 -> 일주일치 시뮬 -> ...

In [5]:
!nvidia-smi

Mon Jun  1 10:41:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [6]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Pretraining, BESTPrediction, PeriodUpdate
import pandas as pd
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from datetime import datetime
import glob
import gc
##############################################################################################################################
path='/content/gdrive/Shared drives/STOCK/version1/dataset/'
save_path='/content/gdrive/Shared drives/STOCK/simulation/models/'
trustval_path='/content/gdrive/Shared drives/STOCK/simulation/trustvalue/'
available_path='/content/gdrive/Shared drives/STOCK/simulation/available/available.csv'
seq_length=50
TARGET=2  #0: 종가 1: 시가 2: 고가 3: 저가
Today_TARGET=0  #0: 종가 1: 시가 2: 고가 3: 저가  -  내일 TARGET 과 비교해서 수익률을 계산할 오늘의 값
fromtime='1996.06.25'
##############################################################################################################################Only for Simulation
import time
SimulationDays=50
restdays=int(SimulationDays)
seed=1000000
UpdatePeriod=14
##############################################################################################################################
def datapreparation(restdays,KEY_path):
    data=pd.read_csv(open(KEY_path, encoding='euc-kr')).iloc[restdays:,:]
    data['null']=np.arange(data.shape[0])
    data=data.set_index('null')
    if int(data.loc[0,'거래량'])==0:          #FaceDivision 처리할 때 index out of range 를 방지하기 위함.
        data.loc[0,'거래량']=1
    if int(data.loc[data.shape[0]-1,'거래량'])==0:
        data.loc[data.shape[0]-1,'거래량']=1
    data.drop(['Unnamed: 0','전일비','날짜'],axis=1,inplace=True)
    data=data.reindex(index=data.index[::-1])

    return data

def FaceDivision(df):
    if np.isnan(df.to_numpy()).any():
        print("there exist NaN in raw data")
        print(df[df==np.nan])
        raise NotImplementedError
    if df.loc[0,'시가']==0:            #거래정지처리
      df.loc[0,'시가']=df.loc[0,'종가']
    
    temp1=pd.DataFrame(df.to_numpy(),columns=df.columns)
    temp2=temp1.drop('거래량',axis=1)
    index1=list(temp1[temp1['거래량']==0].index) 
    #비인기주는 아무도 관심을 주지 않아(...) 거래량이 0인 경우도 있다.
    #또한 액면분할은 여러번 있을 수 있다.
    if len(index1)>1:
      idx=[int(i) for i in range(len(index1)-1) if ( index1[i+1]-index1[i] ) > 1 ]
      idx.insert(0,-1)
      DivisionDays=list(   map(   lambda i:index1[idx[i]+1:idx[i+1]+1] , list(range(len(idx)-1))   )  )
      #DivisionDays=[ index1[idx[i]+1:idx[i+1]+1] for i in range(len(idx-1)) ]
      DivisionDays.append(index1[idx[-1]+1:])
      if len(DivisionDays[0])>0:
        for i in range(len(DivisionDays)):
            beforeDivision=temp2.loc[DivisionDays[i][0]-1,'종가']
            afterDivision=temp2.loc[DivisionDays[i][-1]+1,'시가']
            ratio=afterDivision/beforeDivision
            if 1/ratio>=1.5:                       #after 에 비해 before 가 1.5 배 이상이면 액면분할이라 가정
                temp2.loc[:DivisionDays[i][-1]]=temp2.loc[:DivisionDays[i][-1]]*ratio   #df의 slicing 은 거기까지 포함한다.       

    return temp2

def Emptyfill(df):            #종가가 모두 존재한다는 가정 하에 시가/고가/저가 항목이 비어있으면 채워준다.
    temp=np.array(df.loc[:,'종가'])
    temp=np.where(temp==0,np.nan,temp)
    assert ~np.isnan(temp).any(), "there exist 0 in '종가' column"
    
    for item in ['시가','고가','저가']:
        index1=list(df[df[item].map(int)==0].index)
        for each in index1:
            df.loc[each,item]=df.loc[each,'종가']
    
    return df

def df2nps(df,seqlen):
    seq_len=seqlen+1
    temp=df.to_numpy()
    result=[ temp[i:i+seq_len,:] for i in range(df.shape[0]-seq_len) ]
        
    return np.array(result)

def normalization_and_split(result,train_ratio,TARGET):
    denoms=np.array(list( map(lambda table: [ float(table[0,i]) for i in range(table.shape[1]) ] ,result) ))
    for table in result:
        for i in range(table.shape[1]):
            denominator=float(table[0,i])
            table[:,i]=(table[:,i]/denominator)-1

    #sklearn 에서 자동으로 섞어주는게 있는데 그것보단 순서 고려해서 쪼개는게 좋은듯
    ratio=int(round(len(result)*train_ratio))
    train=result[:ratio,:,:]
    np.random.shuffle(train)
    x_train=train[:,:-1,:]
    y_train=train[:,-1,TARGET]
    
    x_test=result[ratio:,:-1,:]
    y_test=result[ratio:,-1,TARGET]
    denoms=denoms[ratio:,TARGET]

    return x_train, y_train, x_test, y_test, denoms

def trustValue(model,x_test,y_test,denoms):
    pred = model.predict(x_test)
    Ytest=(y_test+1)*denoms
    Ypred=(pred.reshape(pred.shape[0])+1)*denoms
    Ytest_pct=pd.Series(Ytest).pct_change().dropna()
    Ypred_pct=pd.Series(Ypred).pct_change().dropna()
    CORR=np.corrcoef(Ytest_pct.values,Ypred_pct.values)

    return CORR[0,1]

def Prediction(code):
    ###########################################################ImportModel
    model=tf.keras.models.load_model(save_path+code+'.h5')
    ###########################################################ImportTrustValue
    f=open(trustval_path+code+'.txt')
    T=float(  f.readline()  )
    f.close()
    ###########################################################
    data=Emptyfill( FaceDivision( datapreparation(restdays,path+code+'.csv') ) )
    data=data.iloc[data.shape[0]-seq_length:,:].to_numpy()
    Today=list(data[-1,:])
    temp=list(map(lambda i : float(data[0,i]) , list(range(data.shape[1]))  ) )
    for i in range(data.shape[1]):
        denominator=float(data[0,i])
        data[:,i]=(data[:,i]/denominator)-1
    ###########################################################
    data=data.reshape((1,seq_length,4))
    y=model.predict(data)
    TomorrowHigh=(y+1) * temp[TARGET]
    r=float(TomorrowHigh-Today[Today_TARGET])/float(Today[Today_TARGET])
    #def delete_me(obj):
    #    referrers = gc.get_referrers(obj)
    #    for referrer in referrers:
    #        if type(referrer) == dict:
    #            for key, value in referrer.items():
    #                if value is obj:
    #                    referrer[key] = None
    #delete_me(data)
    #delete_me(temp)
    #gc.collect()
    return Today,TomorrowHigh,r,r*T

def PeriodDataPreparation(startP,endP,KEY_path):
    def dayCal(day):
        Y1=int(day[:4])
        M1=int(day[5:7])
        D1=int(day[8:])
        temp1=datetime(Y1,M1,D1)
        Y2=int(fromtime[:4])
        M2=int(fromtime[5:7])
        D2=int(fromtime[8:])
        temp2=datetime(Y2,M2,D2)
        return (temp1-temp2).days

    def PeriodTrue(day):
        if day<=dayCal(endP) and day>=dayCal(startP):
            return True
        else:
            return False
    data=pd.read_csv(open(KEY_path, encoding='euc-kr'))
    index1=data[  data.loc[:,'날짜'].map(dayCal).map(PeriodTrue)  ].index
    data=data.iloc[index1[0]:index1[-1]+seq_length,:]  #start 로부터 49개만 더 필요함
    data['null']=np.arange(data.shape[0])      #뒤에 코딩할 때 다 loc 으로 해놔서 index 다시 밀어줘야 함...시불...
    data=data.set_index('null')
    if int(data.loc[0,'거래량'])==0:          #FaceDivision 처리할 때 index out of range 를 방지하기 위함.
        data.loc[0,'거래량']=1
    if int(data.loc[data.shape[0]-1,'거래량'])==0:
        data.loc[data.shape[0]-1,'거래량']=1
    data.drop(['Unnamed: 0','전일비','날짜'],axis=1,inplace=True)
    data=data.reindex(index=data.index[::-1])

    return data

def Periodnormalization_and_split(result,TARGET):
    for table in result:
        for i in range(table.shape[1]):
            denominator=float(table[0,i])
            table[:,i]=(table[:,i]/denominator)-1
    x_test=result[:,:-1,:]
    y_test=result[:,-1,TARGET]

    return x_test, y_test
##############################################################################################################################주요함수들 - 실사용시 restdays=0 넣으면 될듯?
#Pretraining : 모든 available.csv 에 대해
#BESTPrediction : 모든 model.h5 에 대해
#PeriodUpdate  : 모든 model.h5 에 대해
def Pretraining(restdays):
    KEY_paths=pd.read_csv(open(available_path)).drop('Unnamed: 0',axis=1)
    avail=list(KEY_paths.values.reshape((KEY_paths.shape[0])))
    fail=[]
    with tf.device('/device:GPU:0'):
      for KEY_path in KEY_paths.values:
        try:
          print("Processing with index :",KEY_path[0][-10:-4])
          avail.remove(KEY_path[0])
          data=datapreparation(restdays,KEY_path[0])
          data= FaceDivision(data)
          data=Emptyfill(data)
          #print("Data Length :",data.shape[0])
          result=df2nps( data, seq_length )
          x_train, y_train, x_test, y_test, denoms = normalization_and_split(result,0.9,TARGET)

          model=Sequential()  #LSTM, adam 이 그나마 나음.
          model.add( layers.LSTM(50, return_sequences=True,input_shape=(50,4)) )
          model.add( layers.LSTM(64, return_sequences=False))
          model.add( layers.Dense(1, activation='linear'))
          model.compile(loss='MSE',optimizer='adam')

          history=model.fit(x_train,y_train,epochs=10,batch_size=10,verbose=0)   #여기에서 시간이 오래걸림
          print('Loss at Epoch 10 :',history.history[ 'loss' ][-1])
          if history.history[ 'loss' ][-1]<0.001:
            model.fit(x_train,y_train,epochs=10,batch_size=10,verbose=0)
            T=trustValue(model,x_test,y_test,denoms)
            model.fit(x_test,y_test,epochs=1,batch_size=10,verbose=0)     #신뢰도 구성 후 test 값도 업데이트 해줌.
            model.save(save_path+KEY_path[0][-10:-4]+'.h5')
            print("Correlation Coefficient :",T,"\n")
            f = open(trustval_path+KEY_path[0][-10:-4]+'.txt', "w")
            f.write(str(T))
            f.close()
          else:
            print('loss is not converging. Pass\n')
        except KeyboardInterrupt:
            avail.insert(0,KEY_path[0])
            av=pd.Series(avail)
            av.to_csv(available_path)
            ff=pd.Series(fail)
            thistime=datetime.now()
            ff.to_csv(available_path[:-13]+'ErrorLog '+str(thistime)+'.csv')
            import sys 
            sys.exit(0)
        except:
            print('Error')
            fail.append(KEY_path[0])
            pass
    av=pd.Series(avail)
    av.to_csv(available_path)
    ff=pd.Series(fail)
    thistime=datetime.now()
    ff.to_csv(available_path[:-13]+'ErrorLog '+str(thistime)+'.csv')
    print('ALL MODEL FOR ALL AVAILABLE INDEXES HAS BEEN MADE')

def BESTPrediction(restdays):
    files=glob.glob(save_path[:-1]+'/*.h5')
    codeNtrusted={}
    for codepath in files:
        try:
          code=codepath.replace('\\','/')[-9:-3]
          #print('processing with code :',code)
          with tf.device('/device:GPU:0'):
            _,_,_,trusted=Prediction(code)
          codeNtrusted.setdefault(trusted,[]).append(code)
        except KeyboardInterrupt:
            import sys
            sys.exit(0)
        except:
            print('Prediction about index',code,'failed. Pass')
            pass
    MaximumTrust=max(codeNtrusted.keys())
    codes=codeNtrusted[MaximumTrust]

    TodayEnd=[] #only for simul
    percentages=[]
    PredictHigh=[]

    for code in codes:
        with tf.device('/device:GPU:0'):
          Today,TomorrowHigh,r,trusted=Prediction(code)
        #print("code :",code,"Today :",Today,"Predicted Tomorrow High :",int(TomorrowHigh[0]),'Predicted benefit : {}'.format(r*100),'Corr :',trusted/r)
        TodayEnd.append(int(Today[0]))
        percentages.append(round(r*100,2))
        PredictHigh.append(int(TomorrowHigh[0]))
    return codes,TodayEnd,percentages,PredictHigh


def PeriodUpdate(startP,endP):
    files=glob.glob(save_path[:-1]+'/*.h5')
    for codepath in files:
        try:
            code=codepath.replace('\\','/')[-9:-3]
            #print('processing with code :',code)
            data=PeriodDataPreparation(startP,endP,path+code+'.csv')
            data=FaceDivision(data)
            data=Emptyfill(data)
            #print("Data Length :",data.shape[0])
            result=df2nps( data, seq_length )
            x_test, y_test = Periodnormalization_and_split(result,TARGET)
            model=tf.keras.models.load_model(save_path+code+'.h5')
            with tf.device('/device:GPU:0'):
              model.fit(x_test,y_test,epochs=1,batch_size=1,verbose=0)   #여기에서 시간이 오래걸림
            model.save(save_path+code+'.h5')
        except KeyboardInterrupt:
            import sys
            sys.exit(0)
        except:
            print('Update about index',code,'failed. Pass')
            pass
    print("ALL MODEL UPDATE FOR PERIOD",startP,"TO",endP,"HAS FINISHED")
##############################################################################################################################
if __name__=='__main__': 
    Pretraining(SimulationDays)
    #time.sleep(10)   #구글 드라이브 업데이트가 즉시 되는지 몰라서 일단 넣음
    codes=[]
    NUM=[]    
    Today_temp=[]
    for i in range(SimulationDays+1):
      Today=pd.read_csv(open(path+'005930.csv', encoding='euc-kr')).iloc[restdays,1]
      Today_temp.append(Today)
      print('\n',Today)

      if len(codes)>0 and len(NUM)>0:    #SELL
        for j in range(len(codes)):
          Predicted_Price=PredictHigh[j]
          High_Price=pd.read_csv(open(path+codes[j]+'.csv', encoding='euc-kr')).iloc[restdays,5]
          End_Price=pd.read_csv(open(path+codes[j]+'.csv', encoding='euc-kr')).iloc[restdays,2]   #종가에 파는 것으로 가정. #0: Unnamed 1: 날짜 2: 종가 3: 변화량 4: 시가 5: 고가 6: 저가 7: 거래량
          if Predicted_Price<=High_Price:
            Sell_price=Predicted_Price
          else:
            Sell_price=End_Price
          seed+=NUM[j]*Sell_price
          print("today high :",High_Price," today end :",End_Price)
          print("sold",NUM[j],codes[j],"at price",Sell_price)
          print("restdays :",restdays,"seed :",seed)
        codes=[]
        NUM=[]

      if i<SimulationDays:    #BUY
        codes,TodayEnd,percentages,PredictHigh=BESTPrediction(restdays)
        for percentage in percentages:
          if percentage>2:
            l=len(codes)
            temp=seed/l
            NUM=[]
            for j in range(l):
              num=int(temp/TodayEnd[j])
              NUM.append(num)
              seed-=num*TodayEnd[j]
              print("bought",num,codes[j],"at price",TodayEnd[j]," predicted tomorrow high :",PredictHigh[j])
          else:
             print("nothing to buy today")
      
      if len(Today_temp)==UpdatePeriod:
        PeriodUpdate(Today_temp[0],Today_temp[-1])
        Today_temp=[]

      restdays-=1
      #gc.collect()
    print("Total gain :",round((seed-1000000)/1000000,2)*100,"%")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:241: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:243: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


ALL MODEL FOR ALL AVAILABLE INDEXES HAS BEEN MADE

 2020.02.18
